In [21]:
import boto3
import joblib
import pandas as pd
from prophet import Prophet

s3 = boto3.client('s3')

bucket_name = 'sleep-quality-forecast-bucket'
object_keys = [
                'models/prophet_AHI.pkl',
                'models/prophet_LEAK_95_PERCENTILE.pkl', 
                'models/prophet_MASK_SESSION_COUNT.pkl', 
                'models/prophet_SLEEP_SCORE.pkl', 
                'models/prophet_USAGE_HOURS.pkl'
              ]

In [22]:
results = {}

# Loop through each saved model in S3
for key in object_keys:
    # extract metric name
    metric_name = key.split('/')[-1].replace('prophet_', '').replace('.pkl', '')

    # download model locally
    local_path = f"/tmp/{metric_name}.pkl"
    s3.download_file(bucket_name, key, local_path)

    # load model
    model = joblib.load(local_path)

    # create dates 1 week in future
    future = model.make_future_dataframe(periods=7)
    forecast = model.predict(future)

    # get the last week
    next_week = forecast.tail(7)[['ds', 'yhat_lower', 'yhat', 'yhat_upper']]

    next_week = next_week.rename(columns={
        'ds': 'Date',
        'yhat_lower': 'Lower Estimate',
        'yhat': 'Predicted Value',
        'yhat_upper': 'Upper Estimate'
    })
    
    # save results
    results[metric_name] = next_week

    # print
    print(f"\n7 Day forecast for {metric_name}:")
    print(next_week)
    print("-" * 50)


7 Day forecast for AHI:
          Date  Lower Estimate  Predicted Value  Upper Estimate
852 2025-10-03       -0.041479         0.328244        0.677908
853 2025-10-04       -0.000146         0.369640        0.684636
854 2025-10-05        0.017737         0.359469        0.692633
855 2025-10-06        0.009304         0.369794        0.724602
856 2025-10-07        0.022173         0.371598        0.694419
857 2025-10-08        0.013950         0.338091        0.677960
858 2025-10-09       -0.023840         0.330492        0.660025
--------------------------------------------------

7 Day forecast for LEAK_95_PERCENTILE:
          Date  Lower Estimate  Predicted Value  Upper Estimate
852 2025-10-03        0.525343         5.263238       10.541774
853 2025-10-04       -0.180952         5.095033       10.160593
854 2025-10-05        1.010683         5.645448       10.640857
855 2025-10-06        0.354532         5.210778       10.236330
856 2025-10-07        0.171693         5.043177     

In [23]:
# Save each forecast to S3 as CSV
for metric_name, df_forecast in results.items():
    csv_path = f"/tmp/{metric_name}_forecast.csv"
    df_forecast.to_csv(csv_path, index=False)
    s3.upload_file(csv_path, bucket_name, f"forecasts/{metric_name}_forecast.csv")
    print(f"Uploaded {metric_name}_forecast.csv to s3://{bucket_name}/forecasts/")

Uploaded AHI_forecast.csv to s3://sleep-quality-forecast-bucket/forecasts/
Uploaded LEAK_95_PERCENTILE_forecast.csv to s3://sleep-quality-forecast-bucket/forecasts/
Uploaded MASK_SESSION_COUNT_forecast.csv to s3://sleep-quality-forecast-bucket/forecasts/
Uploaded SLEEP_SCORE_forecast.csv to s3://sleep-quality-forecast-bucket/forecasts/
Uploaded USAGE_HOURS_forecast.csv to s3://sleep-quality-forecast-bucket/forecasts/
